In [ ]:
!pip install --upgrade --quiet ultralytics==8.0.112 clearml tensorboard

In [ ]:
#!pip install opencv-python

!pip install opencv-python==4.8.0.76


In [ ]:
import clearml
clearml.browser_login()

In [ ]:
"""#connect to google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

#!pip install --upgrade --quiet ultralytics==8.0.112 clearml tensorboard

#path = '/content/gdrive/MyDrive/car_detection_training/Car_Detection_Data'
yaml_path = '/content/gdrive/MyDrive/Car_Inspection_Model_Training'"""

"#connect to google drive\nfrom google.colab import drive\ndrive.mount('/content/gdrive', force_remount=True)\n\n#!pip install --upgrade --quiet ultralytics==8.0.112 clearml tensorboard\n\n#path = '/content/gdrive/MyDrive/car_detection_training/Car_Detection_Data'\nyaml_path = '/content/gdrive/MyDrive/Car_Inspection_Model_Training'"

In [ ]:
import boto3
import botocore
import os

s3 = boto3.client("s3")
#s3 = boto3.resource("s3")

In [ ]:
import zipfile

#if not os.path.exists(os.path.dirname(f"./{folder_name}")):
        #os.makedirs(os.path.dirname(f"./{folder_name}"))

with zipfile.ZipFile("/tmp/MRI_Verification_Data.zip", "r") as zip_ref:
    zip_ref.extractall("/tmp/unzipped")
    #zip_ref.extractall(f"./{folder_name}/tmp/unzipped")

In [ ]:
unzipped_files = os.listdir("/tmp/unzipped")
print(unzipped_files)

dest_folder_name = unzipped_files[0]

['MRI_Verification_Data']


In [ ]:
yaml_path = './mri_verification_data.yaml'

In [ ]:
#!pip install torch
#!conda install pytorch

In [ ]:
import cv2
import ultralytics
from ultralytics import YOLO

from datetime import datetime
curr_date = datetime.now().date()
print(curr_date)

2023-10-18


In [ ]:
from clearml import Task

task = Task.init(project_name = "mri_verification_model", task_name = f"mri_verification_model_{curr_date}")

model_variant = "yolov8n"
task.set_parameter("model variant", model_variant)

ClearML Task: overwriting (reusing) task id=03127c5fd712489092a9ed97ca6de336
2023-10-18 19:53:03,174 - clearml.Repository Detection - WARNING - Failed finding Notebook in SageMaker environment. Error is: ''DomainId''
2023-10-18 19:53:03,175 - clearml.Repository Detection - WARNING - Failed accessing the jupyter server(s): [{'base_url': '/', 'hostname': '0.0.0.0', 'password': False, 'pid': 4019, 'port': 8443, 'root_dir': '/home/ec2-user/SageMaker', 'secure': True, 'sock': '', 'token': '', 'url': 'https://ip-172-16-26-33.eu-central-1.compute.internal:8443/', 'version': '2.7.3'}]
2023-10-18 19:53:03,179 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.clear.ml/projects/cd020c0630dc42739e5e574237dbd25c/experiments/03127c5fd712489092a9ed97ca6de336/output/log


In [ ]:
model = YOLO(f"{model_variant}.pt")

args = dict(data= f"{yaml_path}", epochs=20,
            imgsz=640, exist_ok = True, batch = 19, device = 0)

task.connect(args)

{'data': './mri_verification_data.yaml',
 'epochs': 20,
 'imgsz': 640,
 'exist_ok': True,
 'batch': 19,
 'device': 0}

In [ ]:
#train model
results = model.train(**args)
task.close

New https://pypi.org/project/ultralytics/8.0.200 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.112 🚀 Python-3.10.12 torch-2.0.1 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./mri_verification_data.yaml, epochs=20, patience=50, batch=19, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=None, exist_ok=True, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina

<bound method Task.close of <clearml.task.Task object at 0x7fd5398c77c0>>

2023-10-18 20:08:35,194 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/mri_verification_model/mri_verification_model_2023-10-18.03127c5fd712489092a9ed97ca6de336/models/best.pt


In [ ]:
# Load a model
#model = YOLO('yolov8n.pt')  # load an official model
custom_model = YOLO(f'./runs/detect/train/weights/best.pt')  # load a custom model

# Validate the model
metrics = custom_model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category


Ultralytics YOLOv8.0.112 🚀 Python-3.10.12 torch-2.0.1 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients
val: Scanning /home/ec2-user/SageMaker/Alzheimers-Computer-Vision/unzipped/MRI_Verification_Data/valid/labels.cache... 160 images, 0 backgrounds, 0 corrupt: 100%|██████████| 160/160 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  3.52it/s]
                   all        160        160          1          1      0.995      0.995
Speed: 1.9ms preprocess, 4.2ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/val3


array([    0.99498])